In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bingo import APP_NAME
from bingo.custom_logger import getLogger
from bingo.database import SQLiteDatabase

lg = getLogger(APP_NAME)

sqdb = SQLiteDatabase()

try:

    # Optionally, fetch and print all data from the DB after simulations
    lg.info("\n--- All simulation results from database ---")
    cursor = sqdb.connection.cursor()
    cursor.execute("SELECT * FROM simulations")
    rows = cursor.fetchall()
    if rows:
        # Get column names for better DataFrame representation
        col_names = [description[0] for description in cursor.description]
        df = pd.DataFrame(rows, columns=col_names)
        lg.info(f"extracted {df.shape=}")
    else:
        lg.info("No simulation data found in the database.")

finally:
    if sqdb.connection:
        sqdb.connection.close()
        lg.info("Database connection closed.")

In [ ]:
dfraw = df.copy()
dfraw

In [ ]:
df = dfraw.pivot_table(
    values="winning_boards_count",
    index=["board_size", "number_pool_size", "num_boards", "winning_number_size"],
    aggfunc=["mean", "std", "count"],
)

df

In [ ]:
df = dfraw[dfraw["board_size"] == 7]
df = df[df["num_boards"] == 250]
df = df.reset_index()
df = df.pivot_table(
    values="winning_boards_count",
    index=["board_size", "number_pool_size", "num_boards", "winning_number_size"],
    aggfunc=["mean", "std", "count"],
).reset_index()

newcols = []
for col_a, col_b in df.columns:
    if col_b == "":
        newcols.append(col_a)
    else:
        newcols.append(f"{col_b}_{col_a}")
df.columns = newcols
x = df["winning_number_size"]
y = df["number_pool_size"]
z = df["winning_boards_count_mean"]


custom_colorscale = [
    [0, "red"],  # Blue for the minimum value
    [20 / z.max(), "greenyellow"],  # Yellow for the middle value
    [1, "darkgreen"],  # Red for the maximum value
]


fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="markers",
            marker=dict(
                color=z,
                colorscale=custom_colorscale,
                colorbar=dict(
                    title="Winning Boards Count Mean",  # Title for the color bar
                    x=1.02,  # Position the color bar slightly to the right of the plot
                    xpad=0,  # No padding on the x-axis
                    thickness=20,  # Thickness of the color bar
                    len=0.75,  # Length of the color bar (fraction of the plot height)
                ),
            ),
            hoverinfo="text",  # Display custom text on hover
            hovertext=[
                f"Winning Number Size: {xi}<br>Number Pool Size: {yi}<br>Winning Boards Count Mean: {zi:.2f}"
                for xi, yi, zi in zip(x, y, z)
            ],
        )
    ]
)

fig.update_layout(
    title="Scatter Plot of Winning Boards Count Mean",
    xaxis_title="Winning Number Size",
    yaxis_title="Number Pool Size",
    xaxis=dict(
        tickmode="linear",  # Ensures ticks are spaced linearly
        dtick=10,  # Sets the tick spacing to 10 units
    ),
)

fig.show()

In [ ]:
fig = px.scatter(df, x="winning_number_size", y="number_pool_size", color='winning_boards_count')
fig.show()

In [ ]:
fig = px.scatter(df, x="winning_number_size", y="winning_boards_count", color='number_pool_size')
fig.show()

In [ ]:
def create_scatter_trace(dfin, num_boards, custom_colorscale, xpad: int = 0):
    df_filtered = dfin[dfin["num_boards"] == num_boards]
    df_filtered = df_filtered.reset_index(drop=True)

    df_pivot = df_filtered.pivot_table(
        values="winning_boards_count",
        index=["board_size", "number_pool_size", "winning_number_size"],
        aggfunc=["mean", "std", "count"],
    ).reset_index()

    newcols = []
    for col_a, col_b in df_pivot.columns:
        if col_b == "":
            newcols.append(col_a)
        else:
            newcols.append(f"{col_b}_{col_a}")
    df_pivot.columns = newcols

    x = df_pivot["winning_number_size"]
    y = df_pivot["number_pool_size"]
    z = df_pivot["winning_boards_count_mean"]
    return go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker=dict(
            color=z,
            colorscale=custom_colorscale,
            colorbar=dict(
                title="Winning Boards Count Mean",
                xpad=xpad,
                thickness=20,
                len=0.75,
            ),
        ),
        hoverinfo="text",
        hovertext=[
            f"Winning Number Size: {xi}<br>Number Pool Size: {yi}<br>Winning Boards Count Mean: {zi:.2f}"
            for xi, yi, zi in zip(x, y, z)
        ],
        name=f"Num of players: {num_boards}",  # Name for the legend (though colorbar is primary)
    )


df = dfraw[dfraw["board_size"] == 7]
df = df.reset_index()
z = df["winning_boards_count"]
custom_colorscale = [
    [0, "red"],
    # Use z.max() for dynamic scaling of the middle color
    [30 / z.max() if z.max() > 0 else 0.5, "greenyellow"],
    [1, "darkgreen"],
]

num_boards = df["num_boards"].unique()
num_boards.sort()
subplot_titles = [f"No. of players: {num}" for num in num_boards]

# Create subplots
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=subplot_titles,
    horizontal_spacing=0.1,
)
xpad = 0
r, c = (1, 1)
for i, nb in enumerate(num_boards):
    trace = create_scatter_trace(df, nb, custom_colorscale, xpad=xpad)
    print(f"{r=}, {c=}, {i=}, {nb=}")
    fig.add_trace(trace, row=r, col=c)
    if i % 2 == 1:
        r += 1
    c += 1
    if c >= 3:
        c = 1

    xpad += 50


# Update overall layout
fig.update_layout(
    title_text="Winning Boards Count Mean by Board Size",
    height=1000,  # Adjust height as needed for better visibility
    showlegend=False,  # Colorbars serve as legends, so no need for a separate legend
)

r, c = (1, 1)
for i, _ in enumerate(num_boards):
    print(f"{r=}, {c=}")
    fig.update_xaxes(
        title_text="Winning Number Size", row=r, col=c, tickmode="linear", dtick=5
    )
    if i % 2 == 1:
        r += 1
    c += 1
    if c >= 3:
        c = 1


fig.show()
fig.write_html("simulation_results.html", include_plotlyjs="cdn")

In [ ]:
num_boards = df["num_boards"].unique()
num_boards.sort()
num_boards

In [ ]:
dfraw.to_csv("simulation_results.csv")

In [ ]:
fig.write_html("simulation_results.html", include_plotlyjs="cdn")